In [25]:
# Librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [7]:
# Diego

df_calendar = pd.read_csv("/Users/diegoma/kaggle/calendar.csv")
df_listings = pd.read_csv("/Users/diegoma/kaggle/listings.csv")
df_neighbourhoods = pd.read_csv("/Users/diegoma/kaggle/neighbourhoods.csv")
df_reviews = pd.read_csv("/Users/diegoma/kaggle/reviews.csv")
df_reviews_det = pd.read_csv("/Users/diegoma/kaggle/reviews_detailed.csv")
df_listings_det = pd.read_csv("/Users/diegoma/kaggle/listings_detailed.csv")

In [12]:
df_calendar.head(1)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,167183,2021-04-15,f,$45.00,$45.00,1.0,5.0


In [31]:
df_listings.head(1)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,6369,"Rooftop terrace room , ensuite bathroom",13660,Simon,Chamartín,Hispanoamérica,40.45724,-3.67688,Private room,60,1,78,2020-09-20,0.58,1,180


In [20]:
df_listings_det.head(1)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6369,https://www.airbnb.com/rooms/6369,20210413152844,2021-04-15,"Rooftop terrace room , ensuite bathroom",Excellent connection with the AIRPORT and EXHI...,NaN,https://a0.muscache.com/pictures/683224/4cc318...,13660,https://www.airbnb.com/users/show/13660,...,10.0,10.0,10.0,NaN,f,1,0,1,0,0.58


In [23]:
len(df_neighbourhoods["neighbourhood_group"].unique())

21

In [24]:
len(df_neighbourhoods["neighbourhood"].unique())

128

In [77]:
df_listings['room_type'].unique()

array(['Private room', 'Entire home/apt', 'Shared room', 'Hotel room'],
      dtype=object)

In [3]:
df_listings = pd.read_csv("/Users/diegoma/kaggle/listings.csv")

In [4]:
df = pd.DataFrame(data = df_listings)

In [4]:
# Modelo de regresion lineal: df_listings
# Target = 'price'

# Preparar dataset
df = df.drop(['name','host_name','last_review'], axis = 1)
df.neighbourhood = df.neighbourhood.astype("category")
df.neighbourhood_group = df.neighbourhood_group.astype("category")
df.room_type = df.room_type.astype("category")

# Dividir train-test
X = df.drop(['price'], axis = 1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size = 0.3, random_state = 123)

In [5]:
df.dtypes

id                                   int64
host_id                              int64
neighbourhood_group               category
neighbourhood                     category
latitude                           float64
longitude                          float64
room_type                         category
price                                int64
minimum_nights                       int64
number_of_reviews                    int64
reviews_per_month                  float64
calculated_host_listings_count       int64
availability_365                     int64
dtype: object

In [47]:
#
#
#
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------

df_listings = pd.read_csv("/Users/diegoma/kaggle/listings.csv")

In [48]:
# Creacion del dataset mejorado numerico para el modelo de regresion lineal

df = pd.DataFrame(data = df_listings)
df = df.set_index('id')
df = df.drop(['name', 'host_id', 'host_name', 'last_review'], axis = 1)

In [49]:
# Transformacion del categorical data en numerico
# Dos opciones:
# 1. valor numerico 1 - len(categoria) para cada variable categorica
# 2. dummie variables (one hot encoding), pero son demasiadas categorias menos para room_type

df['neighbourhood'] = df['neighbourhood'].astype('category')
df['neighbourhood'] = df['neighbourhood'].cat.reorder_categories(df["neighbourhood"].unique(), ordered = True)
df['neighbourhood'] = df['neighbourhood'].cat.codes

df['neighbourhood_group'] = df['neighbourhood_group'].astype('category')
df['neighbourhood_group'] = df['neighbourhood_group'].cat.reorder_categories(df["neighbourhood_group"].unique(), ordered = True)
df['neighbourhood_group'] = df['neighbourhood_group'].cat.codes

# Esto si funciona

In [45]:
# One hot encoding para room_type
df_tool = pd.get_dummies(df['room_type'])
df = df.drop(['room_type'], axis = 1)

df['Entire home/apt'] = pd.Series(df_tool['Entire home/apt'], index=df.index)
df['Hotel room'] = pd.Series(df_tool['Hotel room'], index=df.index)
df['Private room'] = pd.Series(df_tool['Private room'], index=df.index)
df['Shared room'] = pd.Series(df_tool['Shared room'], index=df.index)

df.head(1)

,neighbourhood_group,neighbourhood,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Entire home/apt,Hotel room,Private room,Shared room
id,,,,,,,,,,,,,,
6369,0,0,40.45724,-3.67688,60,1,78,0.58,1,180,0,0,1,0


In [50]:
# Valor numerico categorico para room_type (One-Hot no parece funcionar)
df['room_type'] = df['room_type'].astype('category')
df['room_type'] = df['room_type'].cat.reorder_categories(df["room_type"].unique(), ordered = True)
df['room_type'] = df['room_type'].cat.codes

In [51]:
df = df.dropna()
df.head(1)

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
id,,,,,,,,,,,
6369,0,0,40.45724,-3.67688,0,60,1,78,0.58,1,180


In [52]:
# Correlación e info de todas las variables
# VIF dataframe

vif_data = pd.DataFrame()
vif_data["feature"] = df.columns
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(df.values, i)
                         for i in range(len(df.columns))]

print(vif_data)

                           feature           VIF
0              neighbourhood_group      4.115630
1                    neighbourhood      2.857499
2                         latitude  21500.341869
3                        longitude  21346.279630
4                        room_type      2.784046
5                            price      1.061918
6                   minimum_nights      1.033561
7                number_of_reviews      4.396351
8                reviews_per_month      5.414850
9   calculated_host_listings_count      1.277026
10                availability_365      2.250690


In [53]:
# debemos dropear latitude y longitud (O escalar?)
# Debemos dropear la que peor p-valor tenga y para ello, usamos un modelo temporal

mod = sm.OLS(df.dropna().price, df.dropna().drop(['price'], axis = 1))
res = mod.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:                  price   R-squared (uncentered):                   0.058
Model:                            OLS   Adj. R-squared (uncentered):              0.058
Method:                 Least Squares   F-statistic:                              86.51
Date:                Mon, 08 Nov 2021   Prob (F-statistic):                   6.38e-174
Time:                        16:32:02   Log-Likelihood:                     -1.0657e+05
No. Observations:               13981   AIC:                                  2.132e+05
Df Residuals:                   13971   BIC:                                  2.132e+05
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                                     coef    std err          t      P>|t|      [0.025      0.975]
---------------------

In [54]:
# dropeamos las siguientes variables
df = df.drop(['neighbourhood_group', 'longitude', 'calculated_host_listings_count'], axis=1)

In [55]:
# Separamos en train y test con k=30%

X = df.drop(['price'], axis = 1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size = 0.3, random_state = 123)

print('La longitud de instancias X_train: ' + str(len(X_train)) + '\nLa longitud de instancias X_test: ' + str(len(X_test)) + '\n')
print('La longitud de instancias y_train: ' + str(len(y_train)) + '\nLa longitud de instancias y_test: ' + str(len(y_test)) + '\n')

if((len(X_train) == len(y_train)) & (len(X_test) == len(y_test))):
    print("Todo correcto")

La longitud de instancias X_train: 9786
La longitud de instancias X_test: 4195

La longitud de instancias y_train: 9786
La longitud de instancias y_test: 4195

Todo correcto


In [56]:
# Modelo de regresion lineal

model = LinearRegression()
model.fit(X_train, y_train)

print('coefficient of determination:', model.score(X_train, y_train))
print('intercept:', model.intercept_)
print('slope:', model.coef_)

y_pred = model.predict(X_test)

print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

coefficient of determination: 0.014032371423794987
intercept: 26414.216873124766
slope: [-1.65401507e-01 -6.51713819e+02  9.51726687e+01 -6.87338410e-02
  4.59092231e-01 -2.48062168e+01 -1.03595603e-01]
Mean squared error: 257050.56
Coefficient of determination: 0.01


Podemos concluir que el dataframe que estamos utilizando incluye muy poca informacion